In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dense , MaxPooling2D , Flatten ,Input, AveragePooling2D ,Concatenate , Dropout, BatchNormalization
from tensorflow.keras.models import Sequential , load_model , Model


In [7]:
#Supportive net
#Base Net
#Root Net
#inceptionNet

def inceptionNet(x, filters):
    # (1x1) layer
    layer1 = Conv2D(filters = filters[0], kernel_size = (1, 1), strides = 1, 
                   padding = "same", activation = 'relu')(x) 
   

    #(3x3) layer
    layer2 = Conv2D(filters = filters[1][0], kernel_size = (1, 1), strides = 1, 
                   padding = "same", activation = 'relu')(x)
    layer2 = Conv2D(filters = filters[1][1], kernel_size = (3, 3), strides = 1, 
                   padding = "same", activation = 'relu')(layer2) 
    
    
    
    #(5x5) layer
    layer3 = Conv2D(filters = filters[2][0], kernel_size = (1, 1), strides = 1, 
                   padding = "same", activation = 'relu')(x)
    layer3 = Conv2D(filters = filters[2][1], kernel_size = (5, 5), strides = 1, 
                   padding = "same", activation = 'relu')(layer3) 
    
    
    #(1x1) layer
    layer4 = MaxPooling2D(pool_size = (3, 3), strides= 1, padding = "same")(x)
    layer4 = Conv2D(filters = filters[3], kernel_size = (1, 1), strides = 1, padding  = "same",
                   activation = 'relu')(layer4)
    
    return Concatenate(axis = -1)([layer1, layer2, layer3, layer4])


In [8]:
#Helper Function

def helperFunction(x , name = None):
    layer = AveragePooling2D(pool_size=(5,5) , strides=3,padding="valid")(x)
    
    layer = Conv2D(filters = 128 , kernel_size=(1,1) , strides = 1 , padding = "same" , activation = "relu")(layer)
    
    layer = Flatten()(layer)
    layer = Dense(units=256 , activation="relu")(layer)
    layer = Dropout(0.4)(layer)
    
    layer = Dense(units=100 ,activation = "softmax" , name = name)(layer)
    
    return layer

In [9]:
#Google Net

def googleNet():
    
    #Taking input layer
    inputImage = Input(shape=(224 , 224 , 3))
    
    #layer = 01
    layer = Conv2D(filters = 64 , kernel_size=(7,7) , strides=2 , padding = "same" , activation="relu")(inputImage)
    layer = MaxPooling2D(pool_size = (3,3) , strides = 2 , padding = "same" )(layer)
    layer = BatchNormalization()(layer)
    
    
    #layer = 02
    layer = Conv2D(filters=64 , kernel_size = (1,1) , strides=1 , padding = "same" , activation="relu")(layer)
    layer = Conv2D(filters=192 , kernel_size = (3,3) , strides=1 , padding = "same" , activation="relu")(layer)
    layer = BatchNormalization()(layer)
    layer = MaxPooling2D(pool_size = (3,3) , strides =2 , padding="same")(layer)
    
    #layer = 03
    layer = inceptionNet(layer , [64 , (96,128) , (16 , 32) , 32])
    layer = inceptionNet(layer , [128 , (128 , 192) ,(32 , 96) , 64])
    layer = MaxPooling2D(pool_size = (3,3) , strides = 2 , padding = "same")(layer)
    
    
    #layer = 04
    layer = inceptionNet(layer , [192 , (96 , 208) , (16 , 48) , 64])
    helper1 = helperFunction(layer , name="helper1")
    
    
    
    layer = inceptionNet(layer , [160 , (112,224) , (24 , 64) , 64])
    layer = inceptionNet(layer , [128 , (128 , 256) , (24,64) , 64])
    layer = inceptionNet(layer , [112 , (144 , 288) , (32,64) , 64])
    helper2 = helperFunction(layer , name = "helper2")
    
    
    layer = inceptionNet(layer ,[256 , (160 , 320) , (32,128) , 128])
    layer = MaxPooling2D(pool_size = (3,3) , strides = 2 , padding = "same")(layer)
    
    
    #layer = 05
    layer = inceptionNet(layer , [256 , (160,320),(32,128) , 128])
    layer = inceptionNet(layer , [384 , (192 , 384) , (48 , 128) , 128])
    layer = AveragePooling2D(pool_size=(7,7) , strides=1 , padding="same")(layer)
    
    
    #layer = 06
    layer = Flatten()(layer)
    layer = Dropout(0.4)(layer)
    layer = Dense(units = 256 , activation= "linear")(layer)
    final = Dense(units=100 , activation="softmax" , name= "final")(layer)
    
    model = Model(inputs = inputImage , outputs = [final , helper1 ,helper2])
    
    return model
  
    


In [10]:
model = googleNet()

In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_9 (Conv2D)              (None, 112, 112, 64  9472        ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 max_pooling2d_3 (MaxPooling2D)  (None, 56, 56, 64)  0           ['conv2d_9[0][0]']               
                                                                                              